# Social Media Data Explorer

We can create a few apps that allow us to easily explore social media data.

<div class="alert alert-warning" role="alert">
  This notebook uses a slightly newer and improved version of `social_media` that has improvements in the `num_persons` extraction.
</div>

## Prerequisites

### Imports

In [1]:
from apps.table_apps import table_app
from apps.geovis_apps import geo_vis_cluster_app, geo_vis_shapes_app
from IPython.display import display, HTML
import pandas as pd
import numpy as np

### Styling

In [2]:
display(HTML("<style>.container { width:80% !important; }</style>"))
pd.set_option('display.max_colwidth', -1)

## Data

In [3]:
df = pd.read_csv('HUMAN_public_social_media.csv')
df.set_index('uniqueid', inplace=True)
df = df.replace('[]', np.nan)

## Table App

App for exploring raw data.

In [4]:
table_app(df)

## GeoVis App Shapes

Origin and Destination in two layers.

In [ ]:
geo_vis_shapes_app(df)

## GeoVis App Clusters

Clustered single tweets, tooltips show information about tweet (customizable below).

In [ ]:
geo_vis_cluster_app(df)